# Instalamos Dependencias requeridas para esta DEMO
## Documentacion
## https://docs.pola.rs/user-guide/installation/#all 

In [ ]:
!pip install polars[all] holidays SQLAlchemy python-dotenv connectorx fastapi[all]

# Importamos Polars como la libreria de manipulacion de datos y ETL (Usamos esta libreria por que su interfaz se parece a PySpark) 

In [1]:
import polars as pl
import holidays
import datetime as dt

# Cargamos datos crudos desde un storage arbitrario ( Puede ser CSV , una base de datos sql , un archivo parquet , etc)

In [2]:
df = pl.read_csv(source="./storage_sources/customer_segmentation_data.csv")

## Siempre es bueno revisar que hay en el dataframe ( Nro Columnas ,tipos de datos , Nombres de columna ,etc )

In [3]:
df

Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
i64,i64,str,str,str,str,str,i64,str,str,str,str,i64,i64,str,str,str,str,str,str
84966,23,"""Female""","""Married""","""Associate Degr…","""Mizoram""","""Entrepreneur""",70541,"""policy5""","""04-10-2018""","""Phone""","""policy2""",366603,2749,"""Group""","""Email""","""In-Person Meet…","""Afternoon""","""English""","""Segment5"""
95568,26,"""Male""","""Widowed""","""Doctorate""","""Goa""","""Manager""",54168,"""policy5""","""11-06-2018""","""Chat""","""policy1""",780236,1966,"""Group""","""Mail""","""In-Person Meet…","""Morning""","""French""","""Segment5"""
10544,29,"""Female""","""Single""","""Associate Degr…","""Rajasthan""","""Entrepreneur""",73899,"""policy5""","""06-05-2021""","""Email""","""policy3""",773926,4413,"""Group""","""Email""","""Mail""","""Evening""","""German""","""Segment3"""
77033,20,"""Male""","""Divorced""","""Bachelor's Deg…","""Sikkim""","""Entrepreneur""",63381,"""policy5""","""09-02-2018""","""Chat""","""policy2""",787815,4342,"""Family""","""Text""","""In-Person Meet…","""Anytime""","""French""","""Segment3"""
88160,25,"""Female""","""Separated""","""Bachelor's Deg…","""West Bengal""","""Manager""",38794,"""policy1""","""09-10-2018""","""Chat""","""policy4""",366506,1276,"""Family""","""Email""","""Text""","""Weekends""","""English""","""Segment2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
44809,35,"""Female""","""Divorced""","""Associate Degr…","""Andaman and Ni…","""Salesperson""",120850,"""policy3""","""01-01-2019""","""Mobile App""","""policy1""",586401,4404,"""Family""","""In-Person Meet…","""Phone""","""Afternoon""","""German""","""Segment5"""
65485,61,"""Male""","""Single""","""Doctorate""","""Himachal Prade…","""Entrepreneur""",122309,"""policy5""","""5/18/2021""","""Mobile App""","""policy4""",637733,1285,"""Group""","""Text""","""Mail""","""Afternoon""","""German""","""Segment1"""
26213,25,"""Male""","""Divorced""","""Doctorate""","""Assam""","""Teacher""",49258,"""policy2""","""11/27/2018""","""In-Person""","""policy1""",631057,4407,"""Individual""","""Text""","""Text""","""Weekends""","""French""","""Segment4"""


## Si encontramos fechas que estan en formato str siempre es buena idea convertirlos a tipo de dato date

In [4]:
if df.schema["Purchase History"] != pl.Date:
    print("converted")
    df = df.with_columns(
        pl.col("Purchase History")
        .str.replace_all(
            "/",
            "-",
        )
        .str.to_date("%m-%d-%Y")
    )

converted


In [5]:
df["Purchase History"].dtype

Date

In [6]:
df

Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group
i64,i64,str,str,str,str,str,i64,str,date,str,str,i64,i64,str,str,str,str,str,str
84966,23,"""Female""","""Married""","""Associate Degr…","""Mizoram""","""Entrepreneur""",70541,"""policy5""",2018-04-10,"""Phone""","""policy2""",366603,2749,"""Group""","""Email""","""In-Person Meet…","""Afternoon""","""English""","""Segment5"""
95568,26,"""Male""","""Widowed""","""Doctorate""","""Goa""","""Manager""",54168,"""policy5""",2018-11-06,"""Chat""","""policy1""",780236,1966,"""Group""","""Mail""","""In-Person Meet…","""Morning""","""French""","""Segment5"""
10544,29,"""Female""","""Single""","""Associate Degr…","""Rajasthan""","""Entrepreneur""",73899,"""policy5""",2021-06-05,"""Email""","""policy3""",773926,4413,"""Group""","""Email""","""Mail""","""Evening""","""German""","""Segment3"""
77033,20,"""Male""","""Divorced""","""Bachelor's Deg…","""Sikkim""","""Entrepreneur""",63381,"""policy5""",2018-09-02,"""Chat""","""policy2""",787815,4342,"""Family""","""Text""","""In-Person Meet…","""Anytime""","""French""","""Segment3"""
88160,25,"""Female""","""Separated""","""Bachelor's Deg…","""West Bengal""","""Manager""",38794,"""policy1""",2018-09-10,"""Chat""","""policy4""",366506,1276,"""Family""","""Email""","""Text""","""Weekends""","""English""","""Segment2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
44809,35,"""Female""","""Divorced""","""Associate Degr…","""Andaman and Ni…","""Salesperson""",120850,"""policy3""",2019-01-01,"""Mobile App""","""policy1""",586401,4404,"""Family""","""In-Person Meet…","""Phone""","""Afternoon""","""German""","""Segment5"""
65485,61,"""Male""","""Single""","""Doctorate""","""Himachal Prade…","""Entrepreneur""",122309,"""policy5""",2021-05-18,"""Mobile App""","""policy4""",637733,1285,"""Group""","""Text""","""Mail""","""Afternoon""","""German""","""Segment1"""
26213,25,"""Male""","""Divorced""","""Doctorate""","""Assam""","""Teacher""",49258,"""policy2""",2018-11-27,"""In-Person""","""policy1""",631057,4407,"""Individual""","""Text""","""Text""","""Weekends""","""French""","""Segment4"""


In [7]:
nro_filas, nro_columnas = df.shape
print(f"numero de filas = {nro_filas} \nnumero de columnas = {nro_columnas}")

numero de filas = 53504 
numero de columnas = 20


## Casos de uso

### Caso 1 cuando la data obtenida debe ser mezclada con data de otra fuente para poder ser analizada
### En este caso vamos a validar si el purchase history fue extraido en algun feriado de USA

In [8]:
holiday_us = holidays.country_holidays(
    country="US",
    years=[
        *range(
            df["Purchase History"].min().year, df["Purchase History"].max().year + 1
        ),
    ],
)

In [9]:
df_holiday = pl.DataFrame(
    data=list(holiday_us.items()),
    schema=["HOLIDAY_DATE", "HOLIDAY_NAME"],
    orient="row",
)

In [11]:
joined_df = (
    df.join(
        other=df_holiday,
        left_on=["Purchase History"],
        right_on=["HOLIDAY_DATE"],
        how="left",
    )
    .select(*(pl.col(name) for name in df.columns), pl.col("HOLIDAY_NAME"))
    .sort("Purchase History", descending=True)
)

In [12]:
joined_df.columns

['Customer ID',
 'Age',
 'Gender',
 'Marital Status',
 'Education Level',
 'Geographic Information',
 'Occupation',
 'Income Level',
 'Behavioral Data',
 'Purchase History',
 'Interactions with Customer Service',
 'Insurance Products Owned',
 'Coverage Amount',
 'Premium Amount',
 'Policy Type',
 'Customer Preferences',
 'Preferred Communication Channel',
 'Preferred Contact Time',
 'Preferred Language',
 'Segmentation Group',
 'HOLIDAY_NAME']

## Extraemos la el conjunto agrupado de datos agrupados basandonos en Age y Gender ( Edad y genero ) y ordenados basado en Purchase History
## Notese que aqui se estan usando funciones ventana para no filtra mutliples veces el mismo dataframe

In [13]:
age_gender_purchase_df = (
    joined_df.select(
        pl.col("*"),
        pl.col("Purchase History")
        .rank("ordinal", descending=True)
        .over([pl.col("Age"), pl.col("Gender")])
        .alias("row_num"),
    )
    .filter(
        (pl.col("Age") == 51)
        & (pl.col("Gender") == "Female")
        & (pl.col("HOLIDAY_NAME").is_not_null())
    )
    .sort("Purchase History")
)

age_gender_purchase_df

Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group,HOLIDAY_NAME,row_num
i64,i64,str,str,str,str,str,i64,str,date,str,str,i64,i64,str,str,str,str,str,str,str,u32
4367,51,"""Female""","""Divorced""","""Doctorate""","""Sikkim""","""Manager""",111498,"""policy5""",2018-01-15,"""Mobile App""","""policy4""",980257,3609,"""Individual""","""Email""","""Text""","""Morning""","""Mandarin""","""Segment4""","""Martin Luther …",356
5420,51,"""Female""","""Divorced""","""Associate Degr…","""Lakshadweep""","""Entrepreneur""",56311,"""policy4""",2018-09-03,"""Mobile App""","""policy1""",579925,1916,"""Business""","""Text""","""Email""","""Weekends""","""Mandarin""","""Segment1""","""Labor Day""",315
20638,51,"""Female""","""Divorced""","""Associate Degr…","""Odisha""","""Lawyer""",126166,"""policy4""",2018-11-22,"""Chat""","""policy2""",785007,4026,"""Business""","""Email""","""Mail""","""Afternoon""","""French""","""Segment4""","""Thanksgiving""",303
34009,51,"""Female""","""Divorced""","""Bachelor's Deg…","""Kerala""","""Salesperson""",125836,"""policy2""",2019-10-14,"""In-Person""","""policy5""",674120,836,"""Family""","""In-Person Meet…","""Mail""","""Evening""","""Mandarin""","""Segment5""","""Columbus Day""",258
67146,51,"""Female""","""Divorced""","""High School Di…","""Tamil Nadu""","""Nurse""",74475,"""policy4""",2020-05-25,"""Phone""","""policy5""",309146,1041,"""Family""","""In-Person Meet…","""Text""","""Anytime""","""English""","""Segment1""","""Memorial Day""",200
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2543,51,"""Female""","""Married""","""Associate Degr…","""Manipur""","""Lawyer""",31804,"""policy5""",2022-01-17,"""Chat""","""policy2""",144982,762,"""Group""","""Email""","""Mail""","""Weekends""","""English""","""Segment2""","""Martin Luther …",119
64759,51,"""Female""","""Divorced""","""Doctorate""","""Manipur""","""Lawyer""",29611,"""policy3""",2022-12-25,"""Phone""","""policy3""",883127,4470,"""Group""","""In-Person Meet…","""In-Person Meet…","""Anytime""","""Mandarin""","""Segment5""","""Christmas Day""",56
39119,51,"""Female""","""Married""","""High School Di…","""Uttar Pradesh""","""Salesperson""",76224,"""policy3""",2023-01-01,"""Chat""","""policy5""",759556,3453,"""Group""","""Phone""","""In-Person Meet…","""Weekends""","""English""","""Segment4""","""New Year's Day…",54


### Exportamos el resultado hacia un archivo csv o parquet dependiendo de lo que el cliente requiera 

In [15]:
age_gender_purchase_df.write_csv("./csv/exported_age_gender_purchase_df.csv")
age_gender_purchase_df.write_parquet(
    "./parquet/exported_age_gender_purchase_df.parquet"
)

# EXPERIMENTAL

### Experimental : Cargamos la data extraida a una base de datos

In [16]:
import dotenv
import os

dotenv.load_dotenv()

True

### Se carga la data a una base de datos de docker
### para correr el contenedor en local instalar docker y docker compose 
### Seguidamente ejecutar el siguiente comando

docker compose up -d 


In [19]:
schema = "demo_datalake"
db_user = os.environ["DB_USER"]
db_pass = os.environ["DB_PASS"]
db_host = os.environ["DB_HOST"]
db_port = os.environ["DB_PORT"]


connection_uri = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/santex"

age_gender_purchase_df.write_database(
    table_name=f"{schema}.incoming_age_gender_group",
    connection=connection_uri,
    if_table_exists="append",
)

15

## Se lee la base de datos y se valida que se inserto de manera correcta la data

In [20]:
pl.read_database_uri(
    query="select * from santex.demo_datalake.incoming_age_gender_group",
    uri=connection_uri,
)
# .filter(pl.col("Preferred Communication Channel") == "Text")

Customer ID,Age,Gender,Marital Status,Education Level,Geographic Information,Occupation,Income Level,Behavioral Data,Purchase History,Interactions with Customer Service,Insurance Products Owned,Coverage Amount,Premium Amount,Policy Type,Customer Preferences,Preferred Communication Channel,Preferred Contact Time,Preferred Language,Segmentation Group,HOLIDAY_NAME,row_num
i64,i64,str,str,str,str,str,i64,str,date,str,str,i64,i64,str,str,str,str,str,str,str,i64
4367,51,"""Female""","""Divorced""","""Doctorate""","""Sikkim""","""Manager""",111498,"""policy5""",2018-01-15,"""Mobile App""","""policy4""",980257,3609,"""Individual""","""Email""","""Text""","""Morning""","""Mandarin""","""Segment4""","""Martin Luther …",356
5420,51,"""Female""","""Divorced""","""Associate Degr…","""Lakshadweep""","""Entrepreneur""",56311,"""policy4""",2018-09-03,"""Mobile App""","""policy1""",579925,1916,"""Business""","""Text""","""Email""","""Weekends""","""Mandarin""","""Segment1""","""Labor Day""",315
20638,51,"""Female""","""Divorced""","""Associate Degr…","""Odisha""","""Lawyer""",126166,"""policy4""",2018-11-22,"""Chat""","""policy2""",785007,4026,"""Business""","""Email""","""Mail""","""Afternoon""","""French""","""Segment4""","""Thanksgiving""",303
34009,51,"""Female""","""Divorced""","""Bachelor's Deg…","""Kerala""","""Salesperson""",125836,"""policy2""",2019-10-14,"""In-Person""","""policy5""",674120,836,"""Family""","""In-Person Meet…","""Mail""","""Evening""","""Mandarin""","""Segment5""","""Columbus Day""",258
67146,51,"""Female""","""Divorced""","""High School Di…","""Tamil Nadu""","""Nurse""",74475,"""policy4""",2020-05-25,"""Phone""","""policy5""",309146,1041,"""Family""","""In-Person Meet…","""Text""","""Anytime""","""English""","""Segment1""","""Memorial Day""",200
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2543,51,"""Female""","""Married""","""Associate Degr…","""Manipur""","""Lawyer""",31804,"""policy5""",2022-01-17,"""Chat""","""policy2""",144982,762,"""Group""","""Email""","""Mail""","""Weekends""","""English""","""Segment2""","""Martin Luther …",119
64759,51,"""Female""","""Divorced""","""Doctorate""","""Manipur""","""Lawyer""",29611,"""policy3""",2022-12-25,"""Phone""","""policy3""",883127,4470,"""Group""","""In-Person Meet…","""In-Person Meet…","""Anytime""","""Mandarin""","""Segment5""","""Christmas Day""",56
39119,51,"""Female""","""Married""","""High School Di…","""Uttar Pradesh""","""Salesperson""",76224,"""policy3""",2023-01-01,"""Chat""","""policy5""",759556,3453,"""Group""","""Phone""","""In-Person Meet…","""Weekends""","""English""","""Segment4""","""New Year's Day…",54
